In [1]:
import unirest
import fuzzyTimeRatio as fuzzyTime
import pandas as pd
from __future__ import division, print_function
import numpy as np
import matplotlib.pyplot as plt
import skfuzzy as fuzz

recipes = unirest.get("https://spoonacular-recipe-food-nutrition-v1.p.mashape.com/recipes/541391/information?includeNutrition=true",
  headers={
    "X-Mashape-Key": "uI9T1GTt8nmshcUUWJOjq8TQNGBgp1P9Zffjsn7dAbkmTSDt1k",
    "Accept": "application/json"
  }
)
# present data properly
recipeInfo = recipes.body
# get all of the nutrition information
recipeNutritions = recipeInfo['nutrition']['nutrients']
# create dict where each key is a nutrition and its value is the value of the nutrition
nutritionsDict = {nutrition['title']: nutrition['percentOfDailyNeeds'] for nutrition in recipeNutritions}
all_keys = ['Caffeine', 'Calories', 'Carbohydrates', 'Cholesterol', 'Fat', 'Fiber', 'Protein', 'Saturated Fat', 'Sugar', 'Sodium']
nutrition_vector = [nutritionsDict[nutrition] if nutrition in nutritionsDict else 0 for nutrition in all_keys]
nutrition_vector = np.matrix(nutrition_vector)


# fuzzify ratio
ratio = 0.4
_, ratio_degrees = fuzzyTime.member_of_function(ratio)
ratio_degrees

[0.39999999999999991, 0.60000000000000009, 0.0, 0.0]

In [2]:
import cluster_recipes
_, u = cluster_recipes.testClustering(3, 1.5, [541391])

In [3]:
u

array([[  1.00821608e-01],
       [  8.98929103e-01],
       [  2.49288509e-04]])

In [4]:
ratio_ample = ratio_degrees[0]
ratio_comfortable = ratio_degrees[1]
ratio_hasty = ratio_degrees[2]
ratio_impossible = ratio_degrees[3]

activity_low = u[0]
activity_medium = u[1]
activity_high = u[2]

In [5]:
x_fit = np.arange(0, 1.1, 0.1)
fit_lo = fuzz.trimf(x_fit, [0, 0, 0.5])
fit_md = fuzz.trimf(x_fit, [0, 0.5, 1])
fit_hi = fuzz.trimf(x_fit, [0.5, 1, 1])

In [6]:
activation_rule1 = np.fmin(ratio_impossible, fit_lo)

rule2 = np.fmin(ratio_hasty, np.fmax(activity_medium, activity_high))
activation_rule2 = np.fmin(rule2, fit_md)

activation_rule3 = np.fmin(activity_low, fit_lo)

rule4 = np.fmin(ratio_ample, np.fmax(activity_medium, activity_high))
activation_rule4 = np.fmin(rule4, fit_hi)

rule5 = np.fmin(ratio_comfortable, activity_medium)
activation_rule5 = np.fmin(rule5, fit_md)

rule6 = np.fmin(ratio_comfortable, activity_high)
activation_rule6 = np.fmin(rule6, fit_hi)

In [7]:
# Aggregate all three output membership functions together
aggregated = np.fmax(activation_rule1,
                     np.fmax(activation_rule2, np.fmax(activation_rule3, np.fmax(activation_rule4, np.fmax(activation_rule5, activation_rule6)))))

fit = fuzz.defuzz(x_fit, aggregated, 'centroid')
fit

0.53221399091169019